### Sentiment and Thematic Analysis
#### Quantify review sentiment and identify themes to uncover satisfaction drivers and pain points.
* Sentiment Analysis with distilbert-base-uncased-finetuned-sst-2-english
* Thematic Analysis

#### Load The SCraped and Cleaned Data

In [1]:
# import package 
import pandas as pd
df=pd.read_csv("../data/processed/bank_reviews_clean.csv")
df.head(5)


,review_id,review,rating,date,bank,source
0,5112423d-e618-44ba-ba49-62677cb76cd6,This application is very important and advanta...,5,2025-11-28,Commercial Bank of Ethiopia,Google Play Store
1,bcb34681-1dd4-4781-b400-4393bb10b1d9,why didn't work this app?,1,2025-11-28,Commercial Bank of Ethiopia,Google Play Store
2,c69f051a-00f8-4144-8423-b7ebcd328d2d,The app makes our life easier. Thank you CBE!,5,2025-11-28,Commercial Bank of Ethiopia,Google Play Store
3,f8002d06-b5c5-4ed1-9d51-a9a379304cf8,the most advanced app. but how to stay safe?,5,2025-11-27,Commercial Bank of Ethiopia,Google Play Store
4,81000db5-aa51-467e-826c-fc96160e96a8,Good application,4,2025-11-27,Commercial Bank of Ethiopia,Google Play Store
